In [1]:
import networkx as nx

from networkx.algorithms.cluster import average_clustering, triangles, transitivity
from networkx.algorithms.planarity import check_planarity
from networkx.algorithms.efficiency import global_efficiency, local_efficiency
from networkx.algorithms.smallworld import sigma, omega

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

from joblib import Parallel, delayed
from networkx.drawing.nx_pylab import draw

In [20]:
graphs = nx.read_graph6('../data/exp5/graph9.g6')

In [24]:
def compute_metrics(graphs, n_jobs=2, verbose=3):
    """
    graphs : list
        list of nx graphs
    """
    cols = [
        'total_edges', 
        'density',
        'average_clustering', 
        'global_clustering', 
        'total_triangles',
        'global_efficiency',
        'local_efficiency',
        'is_planar',
    ]
    
    def metrics(graph, cols):
        total_edges = graph.number_of_edges()
        if total_edges != 0:
            density = 2*total_edges / (graph.number_of_nodes() * (graph.number_of_nodes() - 1))
        else:
            density = 0
        acc = average_clustering(graph)
        gcc = transitivity(graph)
        total_triangles = np.sum(list(triangles(graph).values()))
        geff = global_efficiency(graph)
        leff = local_efficiency(graph)
        is_planar = check_planarity(graph)[0]

        series = pd.Series(
            [total_edges, density, acc, gcc, total_triangles, geff, leff, is_planar],
            index=cols
        )
        return series
    
    res = Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(metrics)(g, cols) for g in graphs)
        
    return pd.DataFrame(res, columns=cols)